# Citation:
Rui Song, Dai Liu, Dave Zhenyu Chen, Andreas Festag, Carsten Trinitis, Martin Schulz, Alois C. Knoll:
Federated Learning via Decentralized Dataset Distillation in Resource-Constrained Edge Environments. CoRR abs/2208.11311 (2022)


In [ ]:
from copy import deepcopy
import random
import torch
import os
import numpy as np


from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture

def distill_kip_unit(x_train_raw,
                     y_train_raw,
                     num_dd_epoch,
                     seed,
                     k,
                     lr,
                     threshold,
                     target_sample_size,
                     kernel_model,
                     depth,
                     width):

    _, _, kernel_fn = get_kernel_fn(kernel_model, depth, width, 'ntk')
    KERNEL_FN = jax.jit(functools.partial(kernel_fn, get='ntk'))

    channel_means, channel_stds = get_normalization_data(x_train_raw)
    x_train = normalize(x_train_raw, channel_means, channel_stds)
    y_train = one_hot(y_train_raw, 10)

    _, _, x_init_raw, y_init = class_balanced_sample(k, y_train_raw, x_train_raw, y_train, seed=seed)
    x_init = normalize(x_init_raw, channel_means, channel_stds)
    params_init = {'x': x_init, 'y': y_init}

    opt_state, get_params, update_fn = get_update_functions(params_init, KERNEL_FN, lr)
    params = get_params(opt_state)

    pbar = tqdm(range(1, num_dd_epoch + 1))
    for i in pbar:
        # full batch gradient descent
        _, _, x_target_batch, y_target_batch = class_balanced_sample(target_sample_size, y_train_raw, x_train, y_train)
        opt_state, aux = update_fn(i, opt_state, params, x_target_batch, y_target_batch)
        train_loss, train_acc = aux
        params = get_params(opt_state)
        pbar.set_description('Step: %d ' % i + '|Train Loss: %.4f ' % train_loss + '|Train Acc: %.4f ' % train_acc)

        if train_acc > threshold:
            print("converge at loop ", i)
            break

    return params

class FedClient(object):

    def __init__(self, client_id, dataset_id='MNIST'):
        """
        Client in the federated learning for FedD3
        :param client_id: Id of the client
        :param dataset_id: Dataset name for the application scenario
        """
        # Metadata
        self._id = client_id
        self._dataset_id = dataset_id

        # Following private parameters are defined by dataset.
        self._image_length = -1
        self._image_width = -1
        self._image_channel = -1

        if self._dataset_id == 'MNIST':
            self._image_length = 28
            self._image_width = 28
            self._image_channel = 1

        elif self._dataset_id == 'FashionMNIST':
            self._image_length = 28
            self._image_width = 28
            self._image_channel = 1

        elif self._dataset_id == 'CIFAR10':
            self._image_length = 32
            self._image_width = 32
            self._image_channel = 3

        elif self._dataset_id == 'CIFAR100':
            self._image_length = 32
            self._image_width = 32
            self._image_channel = 3

        elif self._dataset_id == 'SVHN':
            self._image_length = 32
            self._image_width = 32
            self._image_channel = 3
        else:
            print('unexpected dataset!')
            exit(0)

        # Local dataset
        self._train_data = None
        self._test_data = None

        # Local distilled dataset
        self._distill_data = {'x': [], 'y': []}

    def load_train(self, data):
        """
        Client loads the decentralized dataset, it can be Non-IID across clients.
        :param data: Local dataset for training.
        """
        self._train_data = {}
        self._train_data = deepcopy(data)

    def load_test(self, data):
        """
        Client loads the test dataset.
        :param data: Dataset for testing.
        """
        self._test_data = {}
        self._test_data = deepcopy(data)

    def kip_distill(self, k,
                    num_train_steps=2000,
                    seed=0,
                    lr=4e-3,
                    threshold=0.995,
                    target_sample_size=5000):
        """
        The client run the FedD3 with KIP-based instance.
        More details on KIP in the paper: https://arxiv.org/abs/2011.00050
        :param k: Number of the local distilled images, this need to be integral times of number of local classes
        :param num_train_steps: Number of the decentralized distillation steps
        :param seed: Index of the used seed
        :param lr: Learning rate of decentralized dataset distillation
        :param threshold: Accuracy threshold for decentralized dataset distillation, when it is exceeded, the distillation breaks out.
        :param target_sample_size: Batch size for decentralized dataset distillation
        :return: Distilled images from decentralized dataset in this client
        """
        res = []
        print("Client %s " % self._id +
              "starts distilling %d " % len(self._train_data['y']) +
              "data points into %s data points" % k)

        params = distill_kip_unit(
            np.array(self._train_data['x'].squeeze(1).permute(0, 2, 3, 1)),
            np.array(self._train_data['y'].squeeze()), num_dd_epoch=num_train_steps, seed=seed, k=k, lr=lr,
            threshold=threshold,
            target_sample_size=target_sample_size, kernel_model='FC', depth=4, width=1024)

        for data, data_label in zip(params['x'], params['y']):
            data = np.asarray(data).tolist()
            label = data_label.argmax(0)
            label = np.asarray(label).tolist()
            k_data_point = [label, data, k]
            res.append(k_data_point)
            self._distill_data['y'].append(k_data_point[0])
            self._distill_data['x'].append(k_data_point[1])

        self._distill_data['x'] = torch.tensor(self._distill_data['x']).permute(0, 3, 1, 2)
        self._distill_data['y'] = torch.tensor(self._distill_data['y'])

        return res

    @property
    def all_select(self):
        """
        The client uploads all of the original dataset
        :return: All of the original images
        """
        return self._train_data

    def herding_gmm(self, k, num_local_class, i_seed):
        """
        The client run the FedD3 with coreset-based instance.
        :param k: Number of the local distilled images, this need to be integral times of number of local classes
        :param num_local_class: Number of the local classes
        :param i_seed: Index of the used seed
        :return: Distilled images from decentralized dataset in this client
        """
        torch.manual_seed(i_seed)
        random.seed(i_seed)
        np.random.seed(i_seed)
        res = []
        self._train_data['y'] = self._train_data['y'].squeeze()
        self._train_data['x'] = self._train_data['x'].squeeze()
        num_datapoint = int(k / num_local_class)
        cls_set = set()
        for cls in self._train_data['y']:
            cls_set.add(cls.item())

        for cls in cls_set:
            sub_data = []
            indexes = torch.nonzero(self._train_data['y'] == cls)
            indexes = indexes[torch.randperm(indexes.shape[0])]
            for index in indexes:
                sub_data.append(self._train_data['x'][index].numpy().reshape(-1).tolist())

            gm = GaussianMixture(n_components=int(k / num_local_class), random_state=0).fit(sub_data)
            for x_data in gm.means_:
                k_data_point = [cls, np.array(x_data).reshape(1, 28, 28), k]
                res.append(k_data_point)

        return res

    def dbscan(self, k, num_local_class, i_seed):
        """
        The client run the FedD3 with DBSCAN-based instance.
        :param k: Number of the local distilled images, this need to be integral times of number of local classes
        :param num_local_class: Number of the local classes
        :param i_seed: Index of the used seed
        :return: Distilled images from decentralized dataset in this client
        """
        torch.manual_seed(i_seed)
        random.seed(i_seed)
        np.random.seed(i_seed)
        res = []
        self._train_data['y'] = self._train_data['y'].squeeze()
        self._train_data['x'] = self._train_data['x'].squeeze()
        num_datapoint = int(k / num_local_class)
        cls_set = set()
        for cls in self._train_data['y']:
            cls_set.add(cls.item())

        for cls in cls_set:
            sub_data = []
            indexes = torch.nonzero(self._train_data['y'] == cls)
            indexes = indexes[torch.randperm(indexes.shape[0])]
            for index in indexes:
                sub_data.append(self._train_data['x'][index].numpy().reshape(-1).tolist())
            db = DBSCAN(eps=40.5, min_samples=2).fit(sub_data)
            core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
            core_samples_mask[db.core_sample_indices_] = True
            labels = db.labels_

            # Number of clusters in labels, ignoring noise if present.
            n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
            print(n_clusters_)
            n_noise_ = list(labels).count(-1)
            unique_labels = set(labels)
            cluster_centers_ = []
            for k in unique_labels:
                # discard the unclustered points
                if k == -1:
                    continue
                class_member_mask = (labels == k)
                cluster = np.array(sub_data)[class_member_mask & core_samples_mask]
                cluster_centers_.append(np.mean(cluster, axis=0))

            for x_data in cluster_centers_:
                k_data_point = [cls, np.array(x_data).reshape(3, 32, 32), k]
                res.append(k_data_point)

        return res

    def save_distilled_dataset(self, exp_dir='client_models', res_root='results'):
        """
        The client saves the distilled images in corresponding directory
        :param exp_dir: Experiment directory name
        :param res_root: Result directory root for saving the result files
        """
        agent_name = 'clients'
        model_save_dir = os.path.join(res_root, exp_dir, agent_name)
        if not os.path.exists(model_save_dir):
            os.makedirs(model_save_dir)
        torch.save(self._distill_data, os.path.join(model_save_dir, self._id + '_distilled_img.pt'))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np
from collections import OrderedDict

"""
We provide the models, which might be used in the experiments on FedD3, as follows:
    - AlexNet model customized for CIFAR-10 (AlexCifarNet) with 1756426 parameters
    - LeNet model customized for MNIST with 61706 parameters
    - Further ResNet models
    - Further Vgg models
"""


# AlexNet model customized for CIFAR-10 with 1756426 parameters
class AlexCifarNet(nn.Module):
    supported_dims = {32}

    def __init__(self):
        super(AlexCifarNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LocalResponseNorm(4, alpha=0.001 / 9.0, beta=0.75, k=1),
            nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.LocalResponseNorm(4, alpha=0.001 / 9.0, beta=0.75, k=1),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
        )
        self.classifier = nn.Sequential(
            nn.Linear(4096, 384),
            nn.ReLU(inplace=True),
            nn.Linear(384, 192),
            nn.ReLU(inplace=True),
            nn.Linear(192, 10),
        )

    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), 4096)
        out = self.classifier(out)
        return out


# LeNet model customized for MNIST with 61706 parameters
class LeNet(nn.Module):
    supported_dims = {28}

    def __init__(self, num_classes=10, in_channels=1):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 6, 5, padding=2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = F.relu(self.conv1(x), inplace=True)  # 6 x 28 x 28
        out = F.max_pool2d(out, 2)  # 6 x 14 x 14
        out = F.relu(self.conv2(out), inplace=True)  # 16 x 7 x 7
        out = F.max_pool2d(out, 2)   # 16 x 5 x 5
        out = out.view(out.size(0), -1)  # 16 x 5 x 5
        out = F.relu(self.fc1(out), inplace=True)
        out = F.relu(self.fc2(out), inplace=True)
        out = self.fc3(out)

        return out


# Further ResNet models
def generate_resnet(num_classes=10, in_channels=1, model_name="ResNet18"):
    if model_name == "ResNet18":
        model = models.resnet18(pretrained=True)
    elif model_name == "ResNet34":
        model = models.resnet34(pretrained=True)
    elif model_name == "ResNet50":
        model = models.resnet50(pretrained=True)
    elif model_name == "ResNet101":
        model = models.resnet101(pretrained=True)
    elif model_name == "ResNet152":
        model = models.resnet152(pretrained=True)
    model.conv1 = nn.Conv2d(in_channels, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    fc_features = model.fc.in_features
    model.fc = nn.Linear(fc_features, num_classes)

    return model


# Further Vgg models
def generate_vgg(num_classes=10, in_channels=1, model_name="vgg11"):
    if model_name == "VGG11":
        model = models.vgg11(pretrained=False)
    elif model_name == "VGG11_bn":
        model = models.vgg11_bn(pretrained=True)
    elif model_name == "VGG13":
        model = models.vgg11(pretrained=False)
    elif model_name == "VGG13_bn":
        model = models.vgg11_bn(pretrained=True)
    elif model_name == "VGG16":
        model = models.vgg11(pretrained=False)
    elif model_name == "VGG16_bn":
        model = models.vgg11_bn(pretrained=True)
    elif model_name == "VGG19":
        model = models.vgg11(pretrained=False)
    elif model_name == "VGG19_bn":
        model = models.vgg11_bn(pretrained=True)

    # first_conv_layer = [nn.Conv2d(1, 3, kernel_size=3, stride=1, padding=1, dilation=1, groups=1, bias=True)]
    # first_conv_layer.extend(list(model.features))
    # model.features = nn.Sequential(*first_conv_layer)
    # model.conv1 = nn.Conv2d(num_classes, 64, 7, stride=2, padding=3, bias=False)

    fc_features = model.classifier[6].in_features
    model.classifier[6] = nn.Linear(fc_features, num_classes)

    return model


class CNN(nn.Module):
    def __init__(self, num_classes=10, in_channels=1):
        super(CNN, self).__init__()

        self.fp_con1 = nn.Sequential(OrderedDict([
            ('con0', nn.Conv2d(in_channels=in_channels, out_channels=32, kernel_size=3, padding=1)),
            ('relu0', nn.ReLU(inplace=True)),
            ]))

        self.ternary_con2 = nn.Sequential(OrderedDict([
            # Conv Layer block 1
            ('conv1', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1, bias=False)),
            ('norm1', nn.BatchNorm2d(64)),
            ('relu1', nn.ReLU(inplace=True)),
            ('pool1', nn.MaxPool2d(kernel_size=2, stride=2)),

            # Conv Layer block 2
            ('conv2', nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1, bias=False)),
            ('norm2', nn.BatchNorm2d(128)),
            ('relu2', nn.ReLU(inplace=True)),
            ('conv3', nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1, bias=False)),
            ('norm3', nn.BatchNorm2d(128)),
            ('relu3', nn.ReLU(inplace=True)),
            ('pool2', nn.MaxPool2d(kernel_size=2, stride=2)),
            # nn.Dropout2d(p=0.05),

            # Conv Layer block 3
            ('conv3', nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1, bias=False)),
            ('norm3', nn.BatchNorm2d(256)),
            ('relu3', nn.ReLU(inplace=True)),
            ('conv4', nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1, bias=False)),
            ('norm4', nn.BatchNorm2d(256)),
            ('relu4', nn.ReLU(inplace=True)),
            ('pool4', nn.MaxPool2d(kernel_size=2, stride=2)),
        ]))

        self.fp_fc = nn.Linear(4096, num_classes, bias = False)

    def forward(self, x):
        x = self.fp_con1(x)
        x = self.ternary_con2(x)
        x = x.view(x.size(0), -1)
        x = self.fp_fc(x)
        output = F.log_softmax(x, dim=1)
        return output


if __name__ == "__main__":
    model_name_list = ["ResNet18", "ResNet34", "ResNet50", "ResNet101", "ResNet152"]
    for model_name in model_name_list:
        model = generate_resnet(num_classes=10, in_channels=1, model_name=model_name)
        model_parameters = filter(lambda p: p.requires_grad, model.parameters())
        param_len = sum([np.prod(p.size()) for p in model_parameters])
        print('Number of model parameters of %s :' % model_name, ' %d ' % param_len)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 173MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed i

Number of model parameters of ResNet18 :  11175370 


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:01<00:00, 72.0MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of model parameters of ResNet34 :  21283530 


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 70.4MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of model parameters of ResNet50 :  23522250 


Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 197MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Number of model parameters of ResNet101 :  42514378 


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth
100%|██████████| 230M/230M [00:03<00:00, 75.0MB/s]


Number of model parameters of ResNet152 :  58158026 


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import json
from json import JSONEncoder
import pickle

json_types = (list, dict, str, int, float, bool, type(None))

class Recorder(object):
    def __init__(self):
        self.res_list = []
        self.res = {'server': {'iid_accuracy': [], 'train_loss': []},
                    'clients': {'iid_accuracy': [], 'train_loss': []}}

    def load(self, filename, label):
        """
        Load the result files
        :param filename: Name of the result file
        :param label: Label for the result file
        """
        with open(filename) as json_file:
            res = json.load(json_file, object_hook=as_python_object)
        self.res_list.append((res, label))

    def plot(self):
        """
        Plot the testing accuracy and training loss on number of epochs or communication rounds
        """
        fig, axes = plt.subplots(2)
        for i, (res, label) in enumerate(self.res_list):
            axes[0].plot(np.array(res['server']['iid_accuracy']), label=label, alpha=1, linewidth=2)
            axes[1].plot(np.array(res['server']['train_loss']), label=label, alpha=1, linewidth=2)

        for i, ax in enumerate(axes):
            ax.set_xlabel('# of Epochs', size=12)
            if i == 0:
                ax.set_ylabel('Testing Accuracy', size=12)
            if i == 1:
                ax.set_ylabel('Training Loss', size=12)
            ax.legend(prop={'size': 12})
            ax.tick_params(axis='both', labelsize=12)
            ax.grid()

In [ ]:
import os
import json
import pickle
import torch
from torch.utils.data import DataLoader, TensorDataset
from json import JSONEncoder
from tqdm import tqdm

from copy import deepcopy

class PythonObjectEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, json_types):
            return super().default(self, obj)
        return {'_python_object': pickle.dumps(obj).decode('latin-1')}


def as_python_object(dct):
    if '_python_object' in dct:
        return pickle.loads(dct['_python_object'].encode('latin-1'))
    return dct


json_types = (list, dict, str, int, float, bool, type(None))
class FedServer(object):
    def __init__(self, epoch, batch_size, lr, momentum, num_workers, dataset_id='mnist', server_id='server', model_name="LeNet", i_seed=0, test_on_gpu=True):
        """
        Server in the federated learning for FedD3
        :param epoch: Number of total training epochs in the server
        :param batch_size: Batch size for the training in the server
        :param lr: Learning rate for the training in the server
        :param momentum: Learning momentum for the training in the server
        :param num_workers: Number of the workers for the training in the server
        :param dataset_id: Dataset name for the application scenario
        :param server_id: Id of the server
        :param model_name: Machine learning model name for the application scenario
        :param i_seed: Index of the seed used in the experiment
        :param test_on_gpu: Flag: 1: Run testing on GPU after every epoch, otherwise 0.
        """
        data_dict = ['MNIST', 'FashionMNIST', 'CIFAR10', 'SVHN', 'CIFAR100']
        assert dataset_id in data_dict, "The dataset is not present"

        self.test_on_gpu = test_on_gpu

        # Server Properties
        self._id = server_id
        self._dataset_id = dataset_id
        self._model_name = model_name
        self._i_seed = i_seed

        # Training related parameters
        self._epoch = epoch
        self._batch_size = batch_size
        self._lr = lr
        self._momentum = momentum
        self._num_workers = num_workers

        # Global test dataset
        self._test_data = None

        # Global distilled dataset
        self._distill_data = None

        # Following private parameters are defined by dataset.
        self.model = None
        self._image_dim = -1
        self._image_channel = -1

        if self._dataset_id == 'MNIST':
            self._num_class = 10
            self._image_dim = 28
            self._image_channel = 1

        if self._dataset_id == 'FashionMNIST':
            self._num_class = 10
            self._image_dim = 28
            self._image_channel = 1

        elif self._dataset_id == 'SVHN':
            self._num_class = 10
            self._image_dim = 32
            self._image_channel = 3

        elif self._dataset_id == 'EMNIST':
            self._num_class = 27
            self._image_dim = 28
            self._image_channel = 1

        elif self._dataset_id == 'CIFAR10':
            self._num_class = 10
            self._image_dim = 32
            self._image_channel = 3

        elif self._dataset_id == 'CIFAR100':
            self._num_class = 100
            self._image_dim = 32
            self._image_channel = 3

        if self._model_name == "ResNet18":
            self.model = generate_resnet(num_classes=self._num_class, in_channels=self._image_channel ,model_name=model_name)
        elif self._model_name == "ResNet50":
            self.model = generate_resnet(num_classes=self._num_class, in_channels=self._image_channel ,model_name=model_name)
        elif self._model_name == "ResNet152":
            self.model = generate_resnet(num_classes=self._num_class, in_channels=self._image_channel, model_name=model_name)
        elif self._model_name == "LeNet":
            self.model = LeNet(self._num_class, self._image_channel)
        elif self._model_name == "VGG11":
            self.model = generate_vgg(self._num_class, self._image_channel, model_name=model_name)
        elif self._model_name == "VGG11_bn":
            self.model = generate_vgg(self._num_class, self._image_channel, model_name=model_name)
        elif self._model_name == "AlexCifarNet":
            self.model = AlexCifarNet()
        elif self._model_name == "CNN":
            self.model = CNN(self._num_class, self._image_channel)
        else:
            print('Model is not supported')

        # Number of model parameter
        model_parameters = filter(lambda p: p.requires_grad, self.model.parameters())
        self.param_len = sum([np.prod(p.size()) for p in model_parameters])
        print('Number of model parameters of %s :' % model_name, ' %d ' % self.param_len)
        # Recording results
        self.recorder = Recorder()
        # Run on the GPU
        gpu = 0
        self._device = torch.device("cuda:{}".format(gpu) if torch.cuda.is_available() and gpu != -1 else "cpu")

    def load_test(self, data):
        """
        Server loads the test dataset.
        :param data: Dataset for testing.
        """
        self._test_data = {}
        self._test_data = deepcopy(data)

    def load_distill(self, data):
        """
        Server loads the decentralized distilled dataset.
        :param data: Dataset for training.
        """
        self._distill_data = {}
        self._distill_data = deepcopy(data)

    def train(self, exp_dir, res_root='results', i_seed=0):
        """
        Server trains models on the decentralized distilled datasets from networks
        :param exp_dir: Experiment directory name
        :param res_root: Result directory root for saving the result files
        :param i_seed: Index of the used seed
        :return: Loss in the training.
        """
        torch.manual_seed(i_seed)
        np.random.seed(i_seed)
        state_dict_list = []

        # Create the train and test loader
        with torch.no_grad():

            train_x = self._distill_data['x'].type(torch.FloatTensor).squeeze()
            if len(train_x.shape) == 3:
                train_x = train_x.unsqueeze(1)
            train_y = self._distill_data['y'].type(torch.LongTensor).squeeze()

            test_x = self._test_data['x'].type(torch.FloatTensor).squeeze()
            if len(test_x.shape) == 3:
                test_x = test_x.unsqueeze(1)
            test_y = self._test_data['y'].type(torch.FloatTensor).squeeze()

            train_loader = DataLoader(TensorDataset(train_x, train_y), batch_size=self._batch_size, shuffle=True)
            test_loader = DataLoader(TensorDataset(test_x, test_y), batch_size=self._batch_size, shuffle=True)

            self.model.to(self._device)
            # optimizer = torch.optim.SGD(self.model.parameters(), lr=self._lr, momentum=self._momentum)
            optimizer = torch.optim.Adam(self.model.parameters(), lr=self._lr, weight_decay=1e-4)
            loss_func = nn.CrossEntropyLoss()

        # Train process
        pbar = tqdm(range(self._epoch))
        for epoch in pbar:
            for step, (x, y) in enumerate(train_loader):

                with torch.no_grad():
                    b_x = x.to(self._device)  # Tensor on GPU
                    b_y = y.to(self._device)  # Tensor on GPU

                with torch.enable_grad():
                    self.model.train()
                    output = self.model(b_x)
                    loss = loss_func(output, b_y)
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

            # Recording the train loss during the training
            self.recorder.res['server']['train_loss'].append(loss.data.cpu().numpy())

            # Test process
            if self.test_on_gpu:
                accuracy_collector = 0
                for step, (x, y) in enumerate(test_loader):
                    with torch.no_grad():
                        b_x = x.to(self._device)  # Tensor on GPU
                        b_y = y.to(self._device)  # Tensor on GPU
                        test_output = self.model(b_x)
                        pred_y = torch.max(test_output, 1)[1].to(self._device).data.squeeze()
                        accuracy_collector = accuracy_collector + sum(pred_y == b_y)
                accuracy = accuracy_collector / self._test_data['y'].size(0)
                self.recorder.res['server']['iid_accuracy'].append(accuracy.cpu().numpy())

                pbar.set_description('Epoch: %d' % epoch +
                                     '| Train loss: %.4f ' % loss.data.cpu().numpy() +
                                     '| Accuracy: %.4f' % accuracy +
                                     '| Max Acc: %.4f' % np.max(np.array(self.recorder.res['server']['iid_accuracy'])))
            else:
                pbar.set_description('Epoch: %d', epoch + '| Train loss: %.4f ' % loss.data.cpu().numpy())
            state_dict_list.append(self.model.state_dict())

        # Save the results
        if not os.path.exists(res_root):
            os.makedirs(res_root)

        with open(os.path.join(res_root, exp_dir), "w") as jsfile:
            json.dump(self.recorder.res, jsfile, cls=PythonObjectEncoder)

        return loss.data.cpu().numpy()

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import os


def load_data(name, root='./data', download=True, save_pre_data=True):

    data_dict = ['MNIST', 'EMNIST', 'FashionMNIST', 'CelebA', 'CIFAR10', 'QMNIST', 'SVHN', 'CIFAR100']
    assert name in data_dict, "The dataset is not present"

    if not os.path.exists(root):
        os.makedirs(root, exist_ok=True)

    file_dir = root+'/prepared/'
    test_data_file = file_dir + name + '_test.pt'
    train_data_file = file_dir + name + '_train.pt'
    test_targets_file = file_dir + name + '_test_label.pt'
    train_targets_file = file_dir + name + '_train_label.pt'

    if not os.path.exists(file_dir):
        os.makedirs(file_dir, exist_ok=True)

    all_file_there = os.path.exists(train_data_file) and os.path.exists(test_data_file) and os.path.exists(train_targets_file) and os.path.exists(test_targets_file)

    if save_pre_data or not all_file_there:

        if name == 'MNIST':
            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
            trainset = torchvision.datasets.MNIST(root=root, train=True, download=download, transform=transform)
            testset = torchvision.datasets.MNIST(root=root, train=False, download=download, transform=transform)

        elif name == 'EMNIST':
            # byclass, bymerge, balanced, letters, digits, mnist
            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
            trainset = torchvision.datasets.EMNIST(root=root, train=True, split= 'letters', download=download, transform=transform)
            testset = torchvision.datasets.EMNIST(root=root, train=False, split= 'letters', download=download, transform=transform)

        elif name == 'FashionMNIST':
            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
            trainset = torchvision.datasets.FashionMNIST(root=root, train=True, download=download, transform=transform)
            testset = torchvision.datasets.FashionMNIST(root=root, train=False, download=download, transform=transform)

        elif name == 'CelebA':
            # Could not loaded possibly for google drive break downs, try again at week days
            target_transform = transforms.Compose([transforms.ToTensor()])
            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
            trainset = torchvision.datasets.CelebA(root=root, split='train', target_type=list, download=download, transform=transform, target_transform=target_transform)
            testset = torchvision.datasets.CelebA(root=root, split='test', target_type=list, download=download, transform=transform, target_transform=target_transform)

        elif name == 'CIFAR10':
            transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010])])
            trainset = torchvision.datasets.CIFAR10(root=root, train=True, download=download, transform=transform)
            testset = torchvision.datasets.CIFAR10(root=root, train=False, download=download, transform=transform)
            trainset.targets = torch.Tensor(trainset.targets)
            testset.targets = torch.Tensor(testset.targets)

        elif name == 'CIFAR100':
            transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])
            trainset = torchvision.datasets.CIFAR100(root=root, train=True, transform=transform, download=True)
            testset = torchvision.datasets.CIFAR100(root=root, train=False, transform=transform, download=True)
            trainset.targets = torch.Tensor(trainset.targets)
            testset.targets = torch.Tensor(testset.targets)

        elif name == 'QMNIST':
            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
            trainset = torchvision.datasets.QMNIST(root=root, what='train', compat=True, download=download, transform=transform)
            testset = torchvision.datasets.QMNIST(root=root, what='test', compat=True, download=download, transform=transform)

        elif name == 'SVHN':

            transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4376821, 0.4437697, 0.47280442), (0.19803012, 0.20101562, 0.19703614))])
            trainset = torchvision.datasets.SVHN(root=root, split='train', download=download, transform=transform)
            testset = torchvision.datasets.SVHN(root=root, split='test', download=download, transform=transform)
            trainset.targets = torch.Tensor(trainset.labels)
            testset.targets = torch.Tensor(testset.labels)

        end = len(trainset)-1
        copy_burden = 50
        for i,x in tqdm(enumerate(trainset)):
            if i == end:
                temp = torch.cat((temp, torch.unsqueeze(x[0],0)))
                train_data = torch.cat((train_data, temp))
                train_targets = trainset.targets

            elif i%copy_burden != 0:
                temp = torch.cat((temp, torch.unsqueeze(x[0],0)))

            elif i/copy_burden != 0 and i/copy_burden != 1:
                train_data = torch.cat((train_data, temp))
                temp = torch.unsqueeze(x[0], 0)

            elif i/copy_burden == 0:
                temp = torch.unsqueeze(x[0], 0)

            else:
                train_data = temp
                temp = torch.unsqueeze(x[0], 0)

        end = len(testset)-1
        for i,x in tqdm(enumerate(testset)):
            if i == end:
                temp = torch.cat((temp, torch.unsqueeze(x[0],0)))
                test_data = torch.cat((test_data, temp))
                test_targets = testset.targets

            elif i%copy_burden != 0:
                temp = torch.cat((temp, torch.unsqueeze(x[0],0)))

            elif i/copy_burden != 0 and i/copy_burden != 1:
                test_data = torch.cat((test_data, temp))
                temp = torch.unsqueeze(x[0], 0)

            elif i/copy_burden == 0:
                temp = torch.unsqueeze(x[0], 0)

            else:
                test_data = temp
                temp = torch.unsqueeze(x[0], 0)

        torch.save(train_data, train_data_file)
        torch.save(test_data, test_data_file)
        torch.save(train_targets, train_targets_file)
        torch.save(test_targets, test_targets_file)

    elif all_file_there and not save_pre_data:
        train_data = torch.load(train_data_file)
        test_data = torch.load(test_data_file)
        train_targets = torch.load(train_targets_file)
        test_targets = torch.load(test_targets_file)

    # Set len_classes
    len_classes_dict = {
        'MNIST': 10,
        'EMNIST': 27, # ByClass: 62. ByMerge: 814,255 47.Digits: 280,000 10.Letters: 145,600 26.MNIST: 70,000 10.
        'FashionMNIST': 10,
        'CelebA': 0,
        'CIFAR10': 10,
        'CIFAR100': 100,
        'QMNIST': 10,
        'SVHN': 10
    }

    len_classes = len_classes_dict[name]

    return len_classes, train_data, train_targets, test_data, test_targets


def divide_data(num_client=1, num_local_class=10, dataset_name='emnist', i_seed=0):
    torch.manual_seed(i_seed)

    num_classes, train_data, train_targets, test_data, test_targets = load_data(dataset_name, download=True, save_pre_data=False)

    # import pdb; pdb.set_trace()
    if num_local_class == -1:
        num_local_class = num_classes
    assert 0 < num_local_class <= num_classes, "number of local class should smaller than global number of class"

    trainset_config = {'users': [],
                       'user_data': {},
                       'num_samples': []}
    config_division = {}  # Count of the classes for division
    config_class = {}  # Configuration of class distribution in clients
    config_data = {}  # Configuration of data indexes for each class : Config_data[cls] = [0, []] | pointer and indexes

    for i in range(num_client):
        config_class['f_{0:05d}'.format(i)] = []
        for j in range(num_local_class):
            cls = (i+j) % num_classes
            if cls not in config_division:
                config_division[cls] = 1
                config_data[cls] = [0, []]

            else:
                config_division[cls] += 1
            config_class['f_{0:05d}'.format(i)].append(cls)

    for cls in config_division.keys():
        indexes = torch.nonzero(train_targets == cls)
        num_datapoint = indexes.shape[0]
        indexes = indexes[torch.randperm(num_datapoint)]
        num_partition = num_datapoint // config_division[cls]
        for i_partition in range(config_division[cls]):
            if i_partition == config_division[cls] - 1:
                config_data[cls][1].append(indexes[i_partition * num_partition:])
            else:
                config_data[cls][1].append(indexes[i_partition * num_partition: (i_partition + 1) * num_partition])

    for user in tqdm(config_class.keys()):
        user_data_indexes = torch.tensor([])
        for cls in config_class[user]:
            user_data_index = config_data[cls][1][config_data[cls][0]]
            user_data_indexes = torch.cat((user_data_indexes, user_data_index))
            config_data[cls][0] += 1
            # print(len(user_data_indexes))
        user_data = train_data[user_data_indexes.tolist()]
        user_targets = train_targets[user_data_indexes.tolist()]
        trainset_config['users'].append(user)
        trainset_config['user_data'][user] = {'x': user_data, 'y': user_targets}
        trainset_config['num_samples'] = len(user_data)
        # print(trainset_config['user_data'][user].shape)

    test_iid_data = {'x': None, 'y': None}
    test_iid_data['x'] = test_data
    test_iid_data['y'] = test_targets

    return trainset_config, test_iid_data


In [ ]:
#!/usr/bin/env python3
import torch
import random
import numpy as np
import argparse


torch.cuda.empty_cache()


class Args:
    def __init__(self):
        self.sys_n_client = 50
        self.sys_n_local_class = 10
        self.sys_dataset = 'MNIST'
        self.sys_model = 'LeNet'
        self.sys_i_seed = 0
        self.sys_res_root = 'results'
        self.server_n_epoch = 500
        self.server_bs = 50
        self.server_lr = 0.01
        self.server_momentum = 0.9
        self.server_n_worker = 1
        self.client_instance = 'gmm'
        self.client_n_dd = 10
        self.client_instance_lr = 0.004
        self.client_instance_bs = 10
        self.client_instance_max_n_epoch = 300
        self.client_instance_threshold = 0.999



def main(args):
    """
    Main function for FedD3
    """


    mode_list = ["all_select", "kip_distill", "gmm", "dbscan"]
    assert args.client_instance in mode_list, "The mode is not supported"

    dataset_list = ['MNIST', 'CIFAR10', 'FashionMNIST', 'SVHN', 'CIFAR100']
    assert args.sys_dataset in dataset_list, "The dataset is not supported"

    model_list = ["LeNet", 'AlexCifarNet', 'CNN', 'ResNet18', 'ResNet50', "ResNet152"]
    assert args.sys_model in model_list, "The model is not supported"

    # Number of all distilled data points
    num_images = int(args.client_n_dd * args.sys_n_client)

    # Set the experiment name with hyperparameters
    exp_name = '["%s","%s",%d,%d,%d,%d]' % (
        args.sys_dataset, args.sys_model, num_images, args.client_n_dd, args.sys_n_client, args.sys_n_local_class)

    if args.client_instance == "all_select":
        args.sys_n_client = 1
        args.sys_n_local_class = -1

    torch.manual_seed(args.sys_i_seed)
    random.seed(args.sys_i_seed)
    np.random.seed(args.sys_i_seed)

    client_dict = {}
    distill_dataset = {'x': [], 'y': []}

    # Distribute the dataset into each client with respect to number of local classes
    trainset_config, test_iid_data = divide_data(num_client=args.sys_n_client,
                                                 num_local_class=args.sys_n_local_class,
                                                 dataset_name=args.sys_dataset,
                                                 i_seed=args.sys_i_seed)

    # Initialize each client and distill the local data.
    # (Since it is one-shot, initialization does not have to do separately)
    for client_id in trainset_config['users']:
        client_dict[client_id] = FedClient(client_id, dataset_id=args.sys_dataset)
        client_dict[client_id].load_train(trainset_config['user_data'][client_id])

        ret_data = []
        if args.client_instance == "all_select":
            distill_dataset = client_dict[client_id].all_select
        elif args.client_instance == "gmm":
            ret_data = client_dict[client_id].herding_gmm(k=args.client_n_dd, num_local_class=args.sys_n_local_class, i_seed=args.sys_i_seed)
        elif args.client_instance == "dbscan":
            ret_data = client_dict[client_id].dbscan(k=args.client_n_dd, num_local_class=args.sys_n_local_class, i_seed=args.sys_i_seed)
        elif args.client_instance == "kip_distill":
            ret_data = client_dict[client_id].kip_distill(
                args.client_n_dd,
                num_train_steps=args.client_instance_max_n_epoch,
                seed=args.sys_i_seed,
                lr=args.client_instance_lr,
                threshold=args.client_instance_threshold,
                target_sample_size=args.client_instance_bs)
        for k_data_point in ret_data:
            distill_dataset['y'].append(k_data_point[0])
            distill_dataset['x'].append(k_data_point[1])

    if args.client_instance == "all_select":
        distill_dataset['x'] = torch.tensor(distill_dataset['x'])
        distill_dataset['x'] = distill_dataset['x'].squeeze()
        distill_dataset['y'] = torch.tensor(distill_dataset['y'])
    elif args.client_instance == "gmm":
        distill_dataset['x'] = torch.tensor(distill_dataset['x'])
        distill_dataset['y'] = torch.tensor(distill_dataset['y'])
    elif args.client_instance == "dbscan":
        distill_dataset['x'] = torch.tensor(distill_dataset['x'])
        distill_dataset['y'] = torch.tensor(distill_dataset['y'])
    elif args.client_instance == "kip_distill":
        distill_dataset['x'] = torch.tensor(distill_dataset['x'])
        distill_dataset['x'] = distill_dataset['x'].permute(0, 3, 1, 2)
        distill_dataset['y'] = torch.tensor(distill_dataset['y'])

    # Initialize the server in federated learning
    server = FedServer(epoch=args.server_n_epoch,
                       batch_size=args.server_bs,
                       lr=args.server_lr, momentum=args.server_momentum,
                       num_workers=args.server_n_worker,
                       server_id='server',
                       dataset_id=args.sys_dataset,
                       model_name=args.sys_model,
                       i_seed=args.sys_i_seed,
                       test_on_gpu=True)

    # Server loads non-iid test dataset
    server.load_test(test_iid_data)
    # Server collects the decentralized distilled dataset from clients
    server.load_distill(distill_dataset)
    print('Server gets %d images' % len(distill_dataset['y']))
    print('Server starts experiment with  '
          'i_seed=%d' % args.sys_i_seed,
          '| epoch=%d' % args.server_n_epoch,
          '| batch_size=%d' % args.server_bs,
          '| lr=%.4f' % args.server_lr,
          '| momentum=%.4f' % args.server_momentum,
          '| num_workers=%d' % args.server_n_worker,
          '| dataset_id=%s' % args.sys_dataset,
          '| model_name=%s' % args.sys_model)
    # Server trains models
    server.train(exp_name, args.sys_res_root)


if __name__ == "__main__":
    args = Args()
    main(args)

100%|██████████| 9912422/9912422 [00:00<00:00, 253056783.84it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 39266027.17it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 145252166.35it/s]

...


100%|██████████| 4542/4542 [00:00<00:00, 6614766.93it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



60000it [00:31, 1930.82it/s]
10000it [00:02, 4073.79it/s]
100%|██████████| 50/50 [00:00<00:00, 573.21it/s]
<ipython-input-6-9494e22f8fed>:102: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  distill_dataset['x'] = torch.tensor(distill_dataset['x'])


Number of model parameters of LeNet :  61706 
Server gets 500 images
Server starts experiment with  i_seed=0 | epoch=500 | batch_size=50 | lr=0.0100 | momentum=0.9000 | num_workers=1 | dataset_id=MNIST | model_name=LeNet


Epoch: 499| Train loss: 2.3032 | Accuracy: 0.0892| Max Acc: 0.7385: 100%|██████████| 500/500 [03:11<00:00,  2.62it/s]
